# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [15]:
# Question 1
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

sum(square_root_generator(5))


TypeError: 'float' object is not callable

In [ ]:
# Question 2
def square_root_generator(n):
    yield n ** 0.5
        
for value in square_root_generator(13):
    print(value)


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [18]:
# Question 3 
data = [] 
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

for person in people_1():
    data.append(person)

for person in people_2():
    data.append(person)

print(sum(data[2]))


TypeError: 'float' object is not callable

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [21]:
%%capture
!pip install dlt[duckdb]

In [25]:
import duckdb
import dlt

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='generators')
info = generators_pipeline.run(people_1(),
										table_name="users",
										write_disposition="replace")
print(info)

info = generators_pipeline.run(people_2(),
										table_name="users",
										write_disposition="merge",
                                        primary_key="ID")
print(info)


Pipeline dlt_ipykernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/blitz/Coding-Projects/data_engineering/week_3/course/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707593800.157055 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators
The duckdb destination used duckdb:////Users/blitz/Coding-Projects/data_engineering/week_3/course/dlt_ipykernel_launcher.duckdb location to store data
Load package 1707593800.722999 is LOADED and contains no failed jobs


In [29]:
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n http_download table below:")

users = conn.sql("SELECT * FROM users").df()
display(users)
sum = conn.sql("SELECT SUM(age) FROM users").df()
print(sum)


Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users               │
└─────────────────────┘




 http_download table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1707593800.157055,+Ms8NQBEtxCybg,None
1,2,Person_2,27,City_A,1707593800.157055,AWRFO1jBKHPF6w,None
2,3,Person_3,33,City_B,1707593800.722999,zViSBRK0Q/4Vzw,Job_3
3,6,Person_6,36,City_B,1707593800.722999,WobSWo3DrTGjiw,Job_6
4,4,Person_4,34,City_B,1707593800.722999,WWRx78cSDaBOgg,Job_4
5,5,Person_5,35,City_B,1707593800.722999,pFbEMcN1SNZgnw,Job_5
6,8,Person_8,38,City_B,1707593800.722999,slfhB0K/bhfbzQ,Job_8
7,7,Person_7,37,City_B,1707593800.722999,NekeSj7iQPRrQw,Job_7


   sum(age)
0     266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX